In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

<IB connected to 127.0.0.1:7496 clientId=13>

In [11]:
import stock

tsla = stock.StockX(ib, "AAPL")

tsla.get_fundamentals()
tsla.fundamentals.__dict__

{'industry': 'Technology',
 'category': 'Computers',
 'subcategory': 'Computers',
 'primary_etf': '',
 'secondary_etf': '',
 'currency': 'USD',
 'longName': 'APPLE INC',
 'timeZoneId': 'US/Eastern',
 'tradingHours': ('0400', '2000'),
 'liquidHours': ('0930', '1600'),
 'current_price': 246.75,
 'high_52week': 247.24,
 'low_52week': 164.075,
 'pricing_date': '2024-12-09T00:00:00',
 'volume_10day_avg': 45.10546,
 'enterprise_value': 3786706.0,
 'market_cap': 3745248.0,
 'revenue_ttm': 391035.0,
 'ebitda': 134661.0,
 'net_income_ttm': 93736.0,
 'eps_ttm': 6.06992,
 'revenue_per_share': 25.37854,
 'book_value_per_share': 3.76734,
 'cash_per_share': 4.31117,
 'cash_flow_per_share': 6.82635,
 'dividend_per_share': 0.98,
 'gross_margin': 46.20635,
 'operating_margin': 0.0,
 'net_profit_margin': 0.0,
 'revenue_growth_rate': 0.0,
 'eps_growth_rate': 0.0,
 'pe_ratio': 40.81932,
 'price_to_book': 65.76797,
 'price_to_sales': 0.0,
 'employee_count': 164000,
 'forecast': ForecastMetrics(consensus_re

In [6]:
import pandas as pd

df = pd.DataFrame(tsla.fundamentals.industrys)
df

,type,order,code,description
0,TRBC,1,5310101014,Electric (Alternative) Vehicles
1,NAICS,1,336110,Automobile and Light Duty Motor Vehicle Manufa...
2,NAICS,2,441110,New Car Dealers
3,NAICS,3,441120,Used Car Dealers
4,NAICS,4,335312,Motor and Generator Manufacturing
5,NAICS,5,335910,Battery Manufacturing
6,NAICS,6,334413,Semiconductor and Related Device Manufacturing
7,SIC,1,3711,Motor Vehicles And Car Bodies
8,SIC,2,5511,New And Used Car Dealers
9,SIC,3,5521,Used Car Dealers


In [4]:
tsla.get_daily_data()

Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540
2023-12-07,240.80,244.08,236.98,242.91,916952
2023-12-08,242.00,245.27,239.27,243.23,878583
2023-12-11,242.00,245.00,237.45,239.55,819699
2023-12-12,240.25,241.22,233.87,236.48,785221
...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824
2024-11-27,339.76,342.60,326.59,334.40,441879
2024-11-29,336.00,348.50,334.65,345.44,272292


In [5]:
tsla.set_default_daily_ta()
tsla.frame.data

,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50
date,,,,,,,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540,NaN,NaN,NaN,NaN,NaN,1
2023-12-07,240.80,244.08,236.98,242.91,916952,NaN,NaN,NaN,NaN,NaN,1
2023-12-08,242.00,245.27,239.27,243.23,878583,NaN,NaN,NaN,NaN,NaN,1
2023-12-11,242.00,245.00,237.45,239.55,819699,NaN,NaN,NaN,NaN,NaN,1
2023-12-12,240.25,241.22,233.87,236.48,785221,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824,213.43710,269.0762,723829.0,65.76,-34.953698,45
2024-11-27,339.76,342.60,326.59,334.40,441879,214.18910,271.1760,671255.0,48.25,-34.171217,46
2024-11-29,336.00,348.50,334.65,345.44,272292,214.96490,273.2024,605475.7,21.40,-55.028418,47


In [6]:

tsla.compute_daily_filters_with_scores()


,BRK_UP_MA,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,
2023-12-06,False,False,False,False,True,False,1,False
2023-12-07,False,False,False,False,True,False,1,False
2023-12-08,False,False,False,False,True,False,1,False
2023-12-11,False,False,False,False,True,False,1,False
2023-12-12,False,False,False,False,True,False,1,False
...,...,...,...,...,...,...,...,...
2024-11-26,False,False,True,True,False,True,3,False
2024-11-27,False,False,True,True,False,True,3,False
2024-11-29,False,False,True,True,False,True,3,False


In [7]:
ib.disconnect()

In [8]:
from data import historical_data as hd
from ib_insync import *


tdata = hd.get_hist_data('TSLA', "52 weeksAgo", 'now', '1 day')
tdata

Loading data from data/historical_data_store/TSLA_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/TSLA_1_day.csv


,open,high,low,close,volume
date,,,,,
2023-12-06,240.61,246.57,238.85,240.37,1080540
2023-12-07,240.80,244.08,236.98,242.91,916952
2023-12-08,242.00,245.27,239.27,243.23,878583
2023-12-11,242.00,245.00,237.45,239.55,819699
2023-12-12,240.25,241.22,233.87,236.48,785221
...,...,...,...,...,...
2024-11-26,338.00,346.96,335.21,340.17,470824
2024-11-27,339.76,342.60,326.59,334.40,441879
2024-11-29,336.00,348.50,334.65,345.44,272292


In [ ]:
mansfield, roc, combined = analyze_sector(
    etf_symbol='XLE',
    lookback_period="26 weeksAgo",
    mansfield_period=150,
    roc_period=150,
    ma_short=20,
    ma_long=100,
    verbose=True
)

In [17]:

    



# Example Usage
# Assuming `etf_data` and `market_data` are DataFrames with the required columns


from data import historical_data as hd
import compare



etf_data = hd.get_hist_data('XLF', "52 weeksAgo", 'now', '1 day')
market_data = hd.get_hist_data('SPY', "52 weeksAgo", 'now', '1 day')

analysis = compare.SectorAnalysis(etf_data, market_data)
analysis.compute_all(
    mansfield_period=200,  # Period for Mansfield RSI
    roc_period=200,         # Period for MA ROC ratio
    ma_short=50,           # Short MA period
    ma_long=200           # Long MA period
)
result_df = analysis.get_df()
today_metrics = analysis.get_today(verbose=True)
display(today_metrics)
result_df


Loading data from data/historical_data_store/XLF_1_day.csv
Stored data: 249 rows of data
Loading data from data/historical_data_store/XLF_1_day.csv
Loading data from data/historical_data_store/SPY_1_day.csv
Stored data: 250 rows of data
Loading data from data/historical_data_store/SPY_1_day.csv

=== Today's Market Analysis ===

Mansfield RSI: 4.25
MA ROC Ratio: 1.16
Combined Score: 2.71

📈 BULLISH


mansfield_rsi     4.254055
ma_roc_ratio      1.160618
combined_score    2.707337
dtype: float64

,etf_50_ma,etf_200_ma,market_50_ma,market_200_ma,mansfield_rsi,ma_roc_ratio,combined_score
date,,,,,,,
2024-09-25,43.9020,41.07935,551.0202,520.40360,-0.554947,0.000000,-0.277474
2024-09-26,43.9246,41.12345,551.3066,520.96265,-0.521474,-0.003399,-0.262437
2024-09-27,43.9616,41.16730,551.6480,521.50450,0.199075,0.125981,0.162528
2024-09-30,44.0124,41.21035,552.1318,522.04710,0.014426,0.025382,0.019904
2024-10-01,44.0514,41.24860,552.4014,522.52575,0.510626,0.061587,0.286106
2024-10-02,44.0928,41.28670,552.7696,523.01615,0.406175,-0.079116,0.163530
2024-10-03,44.1360,41.32415,553.2764,523.50465,0.084966,-0.144174,-0.029604
2024-10-04,44.1912,41.36600,553.9360,524.00725,1.017609,0.274168,0.645889
2024-10-07,44.2238,41.40340,554.4046,524.47240,0.609759,0.092630,0.351194


In [5]:
etf_data

,open,high,low,close,volume
date,,,,,
2023-12-08,35.97,36.19,35.83,36.14,311722
2023-12-11,36.13,36.42,36.09,36.38,287274
2023-12-12,36.35,36.66,36.29,36.66,298369
2023-12-13,36.69,37.41,36.55,37.41,525548
2023-12-14,37.28,37.71,37.23,37.55,622257
...,...,...,...,...,...
2024-11-27,51.26,51.62,51.18,51.33,289386
2024-11-29,51.51,51.57,51.21,51.26,138172
2024-12-02,51.35,51.49,50.70,50.86,477889


In [3]:
from ib_insync import IB, Stock
import xml.etree.ElementTree as ET

contract = Stock('TSLA', 'SMART', 'USD')

# fundamental_data = ib.reqFundamentalData(contract, 'ReportsFundamentals')
fundamental_data = ib.reqFundamentalData(contract, 'ReportSnapshot')
root = ET.fromstring(fundamental_data)

In [5]:
def print_xml_tree(element, indent=0):
    """Print the XML tree structure with indentation"""
    # Print current element
    print('  ' * indent + f'<{element.tag}>', end='')
    
    # Print attributes if any
    if element.attrib:
        print(f' {element.attrib}', end='')
        
    # Print text content if it exists and isn't whitespace
    if element.text and element.text.strip():
        print(f' = {element.text.strip()}')
    else:
        print()
        
    # Print all children recursively
    for child in element:
        print_xml_tree(child, indent + 1)

print_xml_tree(root)

<ReportSnapshot> {'Major': '1', 'Minor': '0', 'Revision': '1'}
  <CoIDs>
    <CoID> {'Type': 'RepNo'} = C8279
    <CoID> {'Type': 'CompanyName'} = Tesla Inc
    <CoID> {'Type': 'IRSNo'} = 912197729
    <CoID> {'Type': 'CIKNo'} = 0001318605
    <CoID> {'Type': 'OrganizationPermID'} = 5088024644
  <Issues>
    <Issue> {'ID': '1', 'Type': 'C', 'Desc': 'Common Stock', 'Order': '1'}
      <IssueID> {'Type': 'Name'} = Ordinary Shares
      <IssueID> {'Type': 'Ticker'} = TSLA
      <IssueID> {'Type': 'RIC'} = TSLA.O
      <IssueID> {'Type': 'DisplayRIC'} = TSLA.OQ
      <IssueID> {'Type': 'InstrumentPI'} = 67910050
      <IssueID> {'Type': 'QuotePI'} = 72106022
      <Exchange> {'Code': 'NASD', 'Country': 'USA'} = NASDAQ
      <GlobalListingType> = OSR
      <MostRecentSplit> {'Date': '2022-08-25'} = 3.00003
    <Issue> {'ID': '2', 'Type': 'P', 'Desc': 'Preferred Stock', 'Order': '1'}
      <IssueID> {'Type': 'Name'} = Preference Shares Series B
      <IssueID> {'Type': 'InstrumentPI'} = 7447

In [14]:
from ib_insync import IB, Stock, Index, Contract
import xml.etree.ElementTree as ET

contract = Stock('SXLK', 'SMART', 'USD')
contract.secType = 'STK'

fundamental_data = ib.reqFundamentalData(contract, 'ReportSnapshot')
root2 = ET.fromstring(fundamental_data)

print_xml_tree(root2)

TypeError: a bytes-like object is required, not 'list'

In [4]:
from dataclasses import dataclass
import xml.etree.ElementTree as ET

@dataclass
class StockIndustries:
    type: str = ''  # e.g., 'GICS', 'NAICS', 'SIC', 'TRBC'
    order: int = 0
    code: str = ''
    description: str = ''

def extract_industries_from_xml(xml_data: str) -> list[StockIndustries]:
    root = ET.fromstring(xml_data)
    industries = []
    
    for industry in root.findall('.//Industry'):
        industry_type = industry.get('type', '')
        industry_order = int(industry.get('order', '0'))
        industry_code = industry.get('code', '')
        industry_description = industry.text.strip() if industry.text else ''
        
        industries.append(StockIndustries(
            type=industry_type,
            order=industry_order,
            code=industry_code,
            description=industry_description
        ))
    
    return industries

# Example usage
xml_data = """
<IndustryInfo>
    <Industry type="TRBC" order="1" reported="0" code="5310101014" mnem="">Electric (Alternative) Vehicles</Industry>
    <Industry type="NAICS" order="1" reported="0" code="336110" mnem="">Automobile and Light Duty Motor Vehicle Manufacturing</Industry>
    <Industry type="NAICS" order="2" reported="0" code="441110" mnem="">New Car Dealers</Industry>
    <Industry type="NAICS" order="3" reported="0" code="441120" mnem="">Used Car Dealers</Industry>
    <Industry type="NAICS" order="4" reported="0" code="335312" mnem="">Motor and Generator Manufacturing</Industry>
    <Industry type="NAICS" order="5" reported="0" code="335910" mnem="">Battery Manufacturing</Industry>
    <Industry type="NAICS" order="6" reported="0" code="334413" mnem="">Semiconductor and Related Device Manufacturing</Industry>
    <Industry type="SIC" order="1" reported="0" code="3711" mnem="">Motor Vehicles And Car Bodies</Industry>
    <Industry type="SIC" order="2" reported="0" code="5511" mnem="">New And Used Car Dealers</Industry>
    <Industry type="SIC" order="3" reported="0" code="5521" mnem="">Used Car Dealers</Industry>
    <Industry type="SIC" order="4" reported="0" code="3621" mnem="">Motors And Generators</Industry>
    <Industry type="SIC" order="5" reported="0" code="3691" mnem="">Storage Batteries</Industry>
    <Industry type="SIC" order="6" reported="0" code="3674" mnem="">Semiconductors/related Devices</Industry>
</IndustryInfo>
"""
industries = extract_industries_from_xml(xml_data)
for industry in industries:
    print(industry)

StockIndustries(type='TRBC', order=1, code='5310101014', description='Electric (Alternative) Vehicles')
StockIndustries(type='NAICS', order=1, code='336110', description='Automobile and Light Duty Motor Vehicle Manufacturing')
StockIndustries(type='NAICS', order=2, code='441110', description='New Car Dealers')
StockIndustries(type='NAICS', order=3, code='441120', description='Used Car Dealers')
StockIndustries(type='NAICS', order=4, code='335312', description='Motor and Generator Manufacturing')
StockIndustries(type='NAICS', order=5, code='335910', description='Battery Manufacturing')
StockIndustries(type='NAICS', order=6, code='334413', description='Semiconductor and Related Device Manufacturing')
StockIndustries(type='SIC', order=1, code='3711', description='Motor Vehicles And Car Bodies')
StockIndustries(type='SIC', order=2, code='5511', description='New And Used Car Dealers')
StockIndustries(type='SIC', order=3, code='5521', description='Used Car Dealers')
StockIndustries(type='SIC